![UTFSM](https://github.com/tclaudioe/Scientific-Computing-V3/blob/main/images/Departamento%20de%20Inform%C3%A1tica%20cromatica%20negra%404x-8.png?raw=true)
# INF-285 - Computación Científica
# Laboratorio 2 - Contexto
## 24 de septiembre de 2025

En ingeniería, es común encontrarse con problemas modelados por ecuaciones diferenciales ordinarias (EDO) y, más generalmente, por ecuaciones en derivadas parciales (EDP) no lineales.
Estos problemas aparecen, por ejemplo, en la simulación de fenómenos físicos, procesos de transferencia de calor, y muchas otras áreas.

Al trasladar estos modelos al computador, el proceso típico consiste en discretizar el dominio espacial (y, en algunos casos, el temporal).
Como resultado, la EDO o EDP original se transforma en un sistema de ecuaciones, que muchas veces es no lineal.
Este sistema, que típicamente puede tener miles de incógnitas, toma la forma:
$$
\begin{align*}
    \mathbf{F(\mathbf{u})} &= \mathbf{0},
\end{align*}
$$
donde $\mathbf{F}:\mathbb{R}^n\rightarrow\mathbb{R}^n$ es una función vectorial no lineal y $n$ es la cantidad de ecuaciones e incógnitas, es decir, la dimensión del problema.

Comprender cómo formar y resolver estos sistemas no lineales es fundamental para toda simulación numérica realista.
Además, muchas veces la función $\mathbf{F}$ y su matriz Jacobiana no tienen expresión sencilla, lo que plantea desafíos adicionales tanto teóricos como computacionales.

Notar que comunmente se utiliza $\mathbf{x}$ como el vector incógnita, pero en el contexto que se desarrolla este laboratorio es más conveniente denotarlo por $\mathbf{u}$.

## Del Problema Continuo al Sistema No Lineal

Al abordar un modelo matemático como un problema de valor de frontera (o del inglés Boundary Value Problem, BVP), es decir, una ecuación diferencial ordinaria con condiciones de borde, se propone la siguiente formulación:
$$
\begin{align*}
    w(u''(x),u'(x),u(x)) &= f(x), \quad x\in[0,1],\\
    u(0) &= \alpha,\\
    u(1) &= \beta.
\end{align*}
$$
donde $w:\mathbb{R}^3\rightarrow \mathbb{R}$ es una función conocida, $f(x)$ también es una función conocida, y $\alpha$ y $\beta$ son las condiciones de borde entregadas.
Lo que se busca acá es encontrar una función $u(x)$ tal que satisfaga la ecuación diferencial y las condiciones de borde respectivas al mismo tiempo.
Ahora, desde el punto de vista computacional, se trabajará con una versión discretizada de $u(x)$, es decir, el primer paso consiste en discretizar el dominio $x\in[0,1]$, reemplazando la función incognita por un vector de valores en puntos específicos,
$$
\begin{align*}
    \mathbf{u} &=
        \begin{bmatrix}
            u(x_0), & u(x_1), & \dots, & u(x_i), & \dots, & u(x_n)
        \end{bmatrix}\\
        &= \begin{bmatrix}
            u_0, & u_1, & \dots, & u_i, & \dots, & u_n
        \end{bmatrix},
\end{align*}
$$
donde el dominio discretizado corresponde a:
$$
\begin{align*}
    \mathbf{x} &=
        \begin{bmatrix} x_0, & x_1, & \dots, & x_i, & \dots, & x_n \end{bmatrix}\\
        &= \begin{bmatrix}
            0, & \dfrac{1}{h}, & \dots, & \dfrac{i}{h}, & \dots & 1
        \end{bmatrix},
\end{align*}
$$
donde $h=\dfrac{1}{n}$.
Ahora, con la versión discreta de la función que se busca, se requiere también aproximar la primera y segunda derivadas, tal que podamos reemplazar en el BVP anterior y obtener un sistema de ecuaciones no lineales.
Este procedimiento se conoce con el nombre de diferencias finitas y permite aproximar las derivadas mediante operadores algebraicos y, de este modo, transformar el problema original en un sistema de ecuaciones no lineales, al cual se le puede aplicar los métodos conocidos para resolverlo!
Esta discretización no solo transforma el problema diferencial en uno algebraico, sino que también facilita la implementación computacional.
Notar que el foco de este laboratorio es resolver los sistemas de ecuaciones lineales o no lineales entregados, no discretización en diferencias finitas, pero es bueno entender de donde vienen los problemas a tratar.

Si el modelo original es no lineal, el sistema resultante conservará esta característica.
Por ejemplo, discretizar una ecuación que contiene terminos como $u^{3}$, $\exp(u)$ o $\sin (u)$ produce ecuaciones donde las incógnitas aparecen combinadas de manera no lineal.
Esta no linealidad puede deberse tanto a la propia ecuación diferencial como a la dependencia de los coeficientes respecto de la incógnita.
En consecuencia, el sistema resultante no se puede resolver directamente, sino que requiere técnicas iterativas que manejen la interdependencia no lineal entre las variables.

Para este tipo de sistemas, es posible aplicar el método de Newton, por lo cual resulta esencial construir la matriz Jacobiana del sistema, donde sus entradas corresponden a las derivadas parciales de cada ecuación respecto de cada incógnita.
La matriz Jacobiana actúa como una linealización local del sistema en torno a una aproximación actual de la solución, permitiendo modelar el problema no lineal como una secuencia de problemas lineales que se resuelven iterativamente.

La resolución del sistema no lineal implica, entonces, resolver en cada paso de Newton un sistema lineal asociado a la Jacobiana y actualizar iterativamente la solución hasta alcanzar la convergencia.

**Ejemplo:**

Tomemos una ecuación diferencial ordinaria (EDO) no lineal con condiciones de frontera (BVP).
Consideremos la siguiente EDO:
$$
\begin{align*}
    u''(x) - u(x)^2 &= f(x) = -\pi^2 \sin(\pi\, x) - \sin^2(\pi\, x), \quad x\in[0,1],\\
    u(0) &= 0,\\
    u(1) &= 0.
\end{align*}
$$
La cual tiene como **solución algebraica** la siguiente función:
$$
u(x) = \sin(\pi\, x).
$$


---

Si discretizamos el intervalo $[0,1]$ en $n+1$ puntos equidistantes $(h=1/n)$, y denotando $u_i \approx u(x_i)$, con $u_0 = 0$, $u_n = 0$, para cada nodo interno $i = 1, \ldots, n-1$:
$$
\mathbf{F}_i(\mathbf{u}) = \dfrac{u_{i+1} - 2\,u_i + u_{i-1}}{h^2} - u_i^2 - f(x_i),
$$
donde $u_0 = 0$ y $u_{n} = 0$ representan las condiciones de borde.
Es decir, la función $\mathbf{F}_i(\mathbf{u})$ es,
$$
\begin{align*}
   \mathbf{F}_i(\mathbf{u}) &=
    \begin{bmatrix}
       \dfrac{u_{1} - 2\,u_1}{h^2} - u_1^2 - f(x_1)\\
       \dfrac{u_{2} - 2\,u_2 + u_{1}}{h^2} - u_2^2 - f(x_2)\\
       \vdots\\
       \dfrac{u_{i+1} - 2\,u_i + u_{i-1}}{h^2} - u_i^2 - f(x_i)\\
       \vdots\\
       \dfrac{u_{n-1} - 2\,u_{n-2} + u_{n-3}}{h^2} - u_{n-2}^2 - f(x_{n-2})\\
       \dfrac{ - 2\,u_{n-1} + u_{n-2}}{h^2} - u_{n-1}^2 - f(x_{n-1})\\
    \end{bmatrix}
\end{align*}
$$
donde se incluyeron directamente las condiciones de borde indicadas.

El sistema de ecuaciones no lineales se puede resuelver con el método de Newton, construyendo la matriz Jacobiana:
$$
J(\mathbf u)
=
\begin{bmatrix}
-\dfrac{2}{h^2}-2\,u_1 & \dfrac{1}{h^2}      & 0                   & \cdots & 0 \\
\dfrac{1}{h^2}       & -\dfrac{2}{h^2}-2\,u_2 & \dfrac{1}{h^2}      & \cdots & 0 \\
0                    & \ddots              & \ddots             & \ddots & \vdots \\
\vdots               & \cdots              & \dfrac{1}{h^2}      & -\dfrac{2}{h^2}-2\,u_{n-2} & \dfrac{1}{h^2} \\
0                    & \cdots              & 0                  & \dfrac{1}{h^2} & -\dfrac{2}{h^2}-2\,u_{n-1}
\end{bmatrix}
$$

---

- La **solución numérica** se representa como un vector $\mathbf{u} = [u_1,\dots,u_{n-1}]^\top$, que aproxima la función continua en los nodos de la malla.  
- Al comparar con la **solución exacta $u(x)=\sin(\pi x)$**, podemos evaluar el **error en cada paso de Newton** y observar cómo la iteración converge hacia la solución correcta.  
- Este ejemplo muestra cómo problemas **no lineales** generan sistemas de ecuaciones que requieren técnicas iterativas avanzadas, y cómo Newton se convierte en una herramienta clave para resolverlos.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

data = np.load("bvp_nonlinear_steps.npz", allow_pickle=False)
U_hist   = data["U_hist"]
x_nodes  = data["x_nodes"]
u0, un   = float(data["u0"]), float(data["un"])
R_hist   = data["R_hist"]
u_exact  = data["u_exact"]

def plot_step(iteration):
    u_plot = np.zeros_like(x_nodes)
    u_plot[0], u_plot[-1] = u0, un
    u_plot[1:-1] = U_hist[iteration]

    err_L2 = np.linalg.norm(u_plot - u_exact, 2) / np.sqrt(x_nodes.size)

    plt.figure(figsize=(14,5))
    plt.subplot(121)
    plt.plot(x_nodes, u_exact, "k--", lw=2, label="Exacta sin(πx)")
    plt.plot(x_nodes, u_plot, "ro", lw=2, label=f"Num (iter {iteration})")
    plt.xlabel(r"$x$")
    plt.ylabel(r"$u(x)$")
    plt.title(f"Iter {iteration}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(-0.2, 1.2)

    its = np.arange(0, iteration+1)
    norms = R_hist[:iteration+1]
    plt.subplot(122)
    plt.semilogy(its, norms, "--ro", lw=2)
    plt.xlabel("Iteración")
    plt.ylabel(r"$\|F(u)\|_2$")
    plt.title("Convergencia del residuo")
    plt.grid(True, which="both", alpha=0.3)

    plt.tight_layout()
    plt.show()

interact(
    plot_step,
    iteration=IntSlider(min=0, max=U_hist.shape[0]-1, step=1, value=0,
                        description="Iteración:", style={"description_width":"initial"})
)
